In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
from model import seq2seq, test_model_instance, seq2seq_attn

BATCH_SIZE = 64

x_train = np.loadtxt("akshar_sequences//x_train.csv", delimiter=",", dtype=int)
y_train = np.loadtxt("akshar_sequences//y_train.csv", delimiter=",", dtype=int)
x_test = np.loadtxt("akshar_sequences//x_test.csv", delimiter=",", dtype=int)
y_test = np.loadtxt("akshar_sequences//y_test.csv", delimiter=",", dtype=int)
x_val = np.loadtxt("akshar_sequences//x_val.csv", delimiter=",", dtype=int)
y_val = np.loadtxt("akshar_sequences//y_val.csv", delimiter=",", dtype=int)


class SequenceDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __getitem__(self, index):
        x = torch.from_numpy(self.x[index]).long() 
        y = torch.from_numpy(self.y[index]).long() 
        return x, y
    
    def __len__(self):
        return len(self.x)

train_dataset = SequenceDataset(x_train, y_train)
val_dataset = SequenceDataset(x_val, y_val)
test_dataset = SequenceDataset(x_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
from model import seq2seq_attn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

VOCAB_SIZE = 131
MAX_SEQ_SIZE = 28
BATCH_SIZE = 8

config = {
    'input_embedding_size': [32],
    'num_encoder_layers': [1, 2, 3],
    'num_decoder_layers': [1, 2, 3],
    'hidden_layer_size': [128],
    'cell_type_encoder': ['LSTM', 'GRU', 'RNN'],
    'cell_type_decoder': ['LSTM', 'GRU', 'RNN'],
    'bidirectional': [0, 1],
    'dropout': [0, 0.2],
    'teacher_forcing': [0, 0.5, 1]
}

source = torch.randint(low=0, high=VOCAB_SIZE, size=(BATCH_SIZE, MAX_SEQ_SIZE)).to(device)
target = torch.randint(low=0, high=VOCAB_SIZE, size=(BATCH_SIZE, MAX_SEQ_SIZE)).to(device)

model = model.to(device)

model(source, target).shape

torch.Size([4, 128]) 0
torch.Size([4, 128]) 1
torch.Size([4, 128]) 2
torch.Size([4, 128]) 3
torch.Size([4, 128]) 4
torch.Size([4, 128]) 5
torch.Size([4, 128]) 6
torch.Size([4, 128]) 7
torch.Size([4, 128]) 8
torch.Size([4, 128]) 9
torch.Size([4, 128]) 10
torch.Size([4, 128]) 11
torch.Size([4, 128]) 12
torch.Size([4, 128]) 13
torch.Size([4, 128]) 14
torch.Size([4, 128]) 15
torch.Size([4, 128]) 16
torch.Size([4, 128]) 17
torch.Size([4, 128]) 18
torch.Size([4, 128]) 19
torch.Size([4, 128]) 20
torch.Size([4, 128]) 21
torch.Size([4, 128]) 22
torch.Size([4, 128]) 23
torch.Size([4, 128]) 24
torch.Size([4, 128]) 25
torch.Size([4, 128]) 26
torch.Size([4, 128]) 27


torch.Size([4, 28, 131])